In [ ]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import fftpack
%pylab inline
plt.style.use('seaborn-whitegrid')
pylab.rcParams['figure.figsize'] = (22, 6)
import matplotlib
matplotlib.rcParams.update({'font.size': 20})

In [ ]:
df = pd.read_csv('Data/BMS/Rooms and HVAC/RCU middenhal.csv', delimiter=';', encoding='utf-8')

In [ ]:
df_temp = df[['Tijdstip', 'Gemiddelde ruimtetemp [°C] - RCU Middelhal 255 ()']]
df_temp.is_copy = False
df_temp.columns = ['Time', 'Temperature']

In [ ]:
df_temp.is_copy = False
df_temp['Time'] = pd.to_datetime(df_temp['Time'], format="%d-%m-%Y %H:%M.%S")

df_temp['Temperature'] = [str(x).replace(',', '.') for x in df_temp['Temperature']]
df_temp['Temperature'] = pd.to_numeric(df_temp['Temperature'], errors='coerce')

In [ ]:
df_temp.sort_values(['Time'], inplace=True)

In [ ]:
df_temp = df_temp.loc[df_temp['Time'] > datetime.datetime(2017, 10, 23, 9, 0, 0)]
df_temp = df_temp.loc[df_temp['Time'] <= datetime.datetime(2018, 3, 8, 4, 52, 0)]

In [ ]:
plt.plot(df_temp['Time'], df_temp['Temperature'], linestyle=':', marker='.')
plt.title('Original signal in time domain')
plt.xlabel('Time')
plt.ylabel('Temperature (in celsius)')
plt.show()

In [ ]:
demean = lambda df: df - df.mean()
time = pd.DatetimeIndex(df_temp['Time'])
grouped = df_temp.groupby([time.year, time.month, time.day])

In [ ]:
def normalize(row, mean):

    return (row - mean)/mean

def scale(row, mean):

    return (row - mean)

norm_df = pd.DataFrame()

date_list = []
mean_temp_list = []

for name, group in grouped:
    df_sub = pd.DataFrame(group)
    date_list.append(name)
    mean = df_sub['Temperature'].mean()
    mean_temp_list.append(mean)
    df_sub['Temperature_normalized'] = df_sub.apply(lambda row: normalize(row['Temperature'], mean), axis=1)
    df_sub['Temperature_scaled'] = df_sub.apply(lambda row: scale(row['Temperature'], mean), axis=1)
    norm_df = norm_df.append(df_sub)

In [ ]:
x_range =[(m*480)/(60*60*24) for m in range(1, len(norm_df['Temperature_normalized'])+1)] 
plt.plot(x_range, norm_df['Temperature_normalized'], linestyle=':', marker='.')
plt.title('Signal normalized per day')
plt.ylabel('Normalized Temperature')
plt.xlabel('Time (days)')
plt.show()

In [ ]:
mean1 = np.mean(mean_temp_list)
new_mean_list = [(i-mean1)/mean1 for i in mean_temp_list]

In [ ]:
pylab.rcParams['figure.figsize'] = (22, 6)
plt.plot(range(1, len(mean_temp_list)+1), new_mean_list, linestyle=':', marker='.')
plt.title('Mean temperature/ day')
plt.ylabel('Temperature (in celsius)')
plt.xlabel('Time (days)')
plt.show()

In [ ]:
X_temp_norm = np.fft.fft(norm_df['Temperature_normalized'], norm="ortho")
X_temp_scaled = np.fft.fft(norm_df['Temperature_scaled'], norm="ortho")
X_temp_mean = np.fft.fft(new_mean_list, norm="ortho")

In [ ]:
freqs_norm = np.fft.fftfreq(len(norm_df['Temperature_normalized'])) * (1/480)
freqs_scaled = np.fft.fftfreq(len(norm_df['Temperature_scaled'])) * (1/480)
freqs_mean = np.fft.fftfreq(len(mean_temp_list))

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)
plt.plot(freqs_norm, abs(X_temp_norm))
plt.xlabel('Frequency')
plt.title('DFT')

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)
plt.plot(freqs_mean, abs(X_temp_mean))
plt.xlabel('Frequency')
plt.title('DFT')

In [ ]:
abs(X_temp_mean)

In [ ]:
num = len(X_temp_norm)/2
num

In [ ]:
def dft_map(X, Fs, shift=True):
    resolution = float(Fs) / len(X)
    if shift:
        n, Y = dft_shift(X)
    else:
        Y = X
        n = np.arange(0, len(Y))
    f = n * resolution
    return f, Y

In [ ]:
Fs = 1/480
y = X_temp_norm
resolution = Fs/(len(y))
n = np.arange(0, len(y))
f = n * resolution
pylab.rcParams['figure.figsize'] = (10, 7)
f_new = [i*8*10800 for i in f]
half = len(f_new)/2
plt.plot(f_new[:int(half)], abs(y[:int(half)]))
plt.xlabel('Frequency')
plt.title('DFT')

In [ ]:
Fs3 = 1
y3 = X_temp_mean
resolution3 = Fs3/len(y3)
n3 = np.arange(0, len(y3))
f3 = n3 * resolution3
pylab.rcParams['figure.figsize'] = (10, 7)
f_new3 = f3
half3 = len(f_new3)/2
plt.plot(f_new3[:int(half3)], abs(y3[:int(half3)]))
plt.xlabel('Frequency')
plt.title('DFT')

In [ ]:
filtered_y3 = [i if abs(i)>0.025 else 0 for i in y3]
reconstructed3 = np.fft.ifft(filtered_y3, norm="ortho")

In [ ]:
pylab.rcParams['figure.figsize'] = (22, 6)
plt.plot(range(1, len(mean_temp_list)+1), new_mean_list, linestyle=':', marker='.')
plt.title('Mean temperature/ day')
plt.ylabel('Temperature (in celsius)')
plt.xlabel('Time (days)')
plot(range(1, len(mean_temp_list)+1), reconstructed3)
plt.show()

In [ ]:

print(f_new[-1])

In [ ]:
for i in range(len(y[:12587])):
    if abs(y[i])>0.5:
        print(f_new[i])
        print(abs(y[i]))

In [ ]:
60*24*7.5

In [ ]:
[k for k in range(len(y))if abs(y[k]) > 1.5]

In [ ]:
X_temp_norm_5 = [k  if abs(k) > 5 else 0 for k in X_temp_norm]
X_temp_norm_10 = [k  if abs(k) > 10 else 0 for k in X_temp_norm]
X_temp_norm_50 = [k  if abs(k) > 50 else 0 for k in X_temp_norm]
X_temp_norm_100 = [k  if abs(k) > 100 else 0 for k in X_temp_norm]
X_temp_norm_250 = [k  if abs(k) > 250 else 0 for k in X_temp_norm]

In [ ]:
x_orig = np.fft.ifft(X_temp_norm)
x_reconstruct_5 = np.fft.ifft(X_temp_norm_5)
x_reconstruct_10 = np.fft.ifft(X_temp_norm_10)
x_reconstruct_50 = np.fft.ifft(X_temp_norm_50)
x_reconstruct_100 = np.fft.ifft(X_temp_norm_100)
x_reconstruct_250 = np.fft.ifft(X_temp_norm_250)

In [ ]:
pylab.rcParams['figure.figsize'] = (22, 40)
plt.subplots_adjust(wspace=8)
plt.subplot(611)
axes = plt.gca()
axes.set_ylim([-0.1, 0.25])
plt.title('Reconstructed signal')
plt.xticks(rotation=0)
plt.plot(norm_df['Time'], norm_df['Temperature_normalized'], linestyle=':', marker='.')

plt.subplot(612)
axes = plt.gca()
axes.set_ylim([-0.1, 0.25])
plt.title('Reconsructed signal (y>250)')
plt.xticks(rotation=0)
plt.plot(norm_df['Time'], x_reconstruct_250, linestyle=':', marker='.')

plt.subplot(613)
axes = plt.gca()
axes.set_ylim([-0.1, 0.25])
plt.title('Reconsructed signal (y>100)')
plt.xticks(rotation=0)
plt.plot(norm_df['Time'], x_reconstruct_100, linestyle=':', marker='.')

plt.subplot(614)
axes = plt.gca()
axes.set_ylim([-0.1, 0.25])
plt.title('Reconsructed signal (y>50)')
plt.xticks(rotation=0)
plt.plot(norm_df['Time'], x_reconstruct_50, linestyle=':', marker='.')

plt.subplot(615)
axes = plt.gca()
axes.set_ylim([-0.1, 0.25])
plt.title('Reconsructed signal (y>10)')
plt.xticks(rotation=0)
plt.plot(norm_df['Time'], x_reconstruct_10, linestyle=':', marker='.')

plt.subplot(616)
axes = plt.gca()
axes.set_ylim([-0.1, 0.25])
plt.title('Reconsructed signal (y>5)')
plt.xticks(rotation=0)
plt.plot(norm_df['Time'], x_reconstruct_5, linestyle=':', marker='.')

In [ ]:
norm_df.sort_values(['Time'], inplace=True)
norm_df.reset_index(drop=True, inplace=True)

In [ ]:
normalized_y =norm_df['Temperature_normalized'].rolling(window=180).mean()
normalized_y.fillna(0, inplace=True)

In [ ]:
# pylab.rcParams['figure.figsize'] = (22, 8)
# plt.plot(normalized_y)

In [ ]:
# new_t = [(m*480)/(60*60*24) for m in range(len(x_orig))]

In [ ]:
# plt.plot(new_t, x_orig)